Get Data for recent high water levels for Tilghman

Steps:
1. Get the data
2. Load to a dataframe
3. Analyze for daily high water

In [1]:
# closest location Cambridge Maryland
# https://tidesandcurrents.noaa.gov/waterlevels.html?id=8571892

# Link for CSV:
# https://api.tidesandcurrents.noaa.gov/api/prod/datagetter
#  ?product=water_level&application=NOS.COOPS.TAC.WL
#  &begin_date=20250201&end_date=20250202&datum=MLLW
#  &station=8571892&time_zone=GMT&units=english&format=csv

import pandas as pd
import sqlite3
import csv
import requests

In [2]:
conn = sqlite3.connect('water_levels.db')
scur = conn.cursor()

In [3]:
def url(year,month,station):
    first_day_of_month = 1
    short_mths = [9,4,6,11]
    if month == 2:
        last_day_of_month = 28
    elif month in short_mths:
        last_day_of_month = 30
    else:
        last_day_of_month = 31

    url = f"""https://api.tidesandcurrents.noaa.gov/api/prod/datagetter
?product=water_level&application=NOS.COOPS.TAC.WL
&begin_date={year}{month:02d}{first_day_of_month:02d}&end_date={year}{month:02d}{last_day_of_month}&datum=MLLW
&station={station}&time_zone=GMT&units=english&format=csv"""

    return (url.replace('\n',''))

year = 2010
month = 11
station = '8571892'
print(url(year,month,station))

https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?product=water_level&application=NOS.COOPS.TAC.WL&begin_date=20101101&end_date=20101130&datum=MLLW&station=8571892&time_zone=GMT&units=english&format=csv


In [5]:
year = 2024
station = 8571892
for month in range(12,0,-1):
    print(month,url(year,month,station))

12 https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?product=water_level&application=NOS.COOPS.TAC.WL&begin_date=20241201&end_date=20241231&datum=MLLW&station=8571892&time_zone=GMT&units=english&format=csv
11 https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?product=water_level&application=NOS.COOPS.TAC.WL&begin_date=20241101&end_date=20241130&datum=MLLW&station=8571892&time_zone=GMT&units=english&format=csv
10 https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?product=water_level&application=NOS.COOPS.TAC.WL&begin_date=20241001&end_date=20241031&datum=MLLW&station=8571892&time_zone=GMT&units=english&format=csv
9 https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?product=water_level&application=NOS.COOPS.TAC.WL&begin_date=20240901&end_date=20240930&datum=MLLW&station=8571892&time_zone=GMT&units=english&format=csv
8 https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?product=water_level&application=NOS.COOPS.TAC.WL&begin_date=20240801&end_date=2024083

In [131]:
# Download the CSV file
response = requests.get(url(year,month,station))
response.raise_for_status()  # Raise an exception if the download fails


In [132]:
response.text[:2000]

'Date Time, Water Level, Sigma, O or I (for verified), F, R, L, Quality \n2010-11-01 00:00,-0.050,0.007,0,0,0,0,v\n2010-11-01 00:06,-0.011,0.007,0,0,0,0,v\n2010-11-01 00:12,0.025,0.007,0,0,0,0,v\n2010-11-01 00:18,0.068,0.007,0,0,0,0,v\n2010-11-01 00:24,0.114,0.007,0,0,0,0,v\n2010-11-01 00:30,0.153,0.007,0,0,0,0,v\n2010-11-01 00:36,0.189,0.007,0,0,0,0,v\n2010-11-01 00:42,0.229,0.007,0,0,0,0,v\n2010-11-01 00:48,0.265,0.007,0,0,0,0,v\n2010-11-01 00:54,0.301,0.007,0,0,0,0,v\n2010-11-01 01:00,0.337,0.007,0,0,0,0,v\n2010-11-01 01:06,0.373,0.003,0,0,0,0,v\n2010-11-01 01:12,0.406,0.003,0,0,0,0,v\n2010-11-01 01:18,0.439,0.003,0,0,0,0,v\n2010-11-01 01:24,0.468,0.007,0,0,0,0,v\n2010-11-01 01:30,0.508,0.007,0,0,0,0,v\n2010-11-01 01:36,0.540,0.007,0,0,0,0,v\n2010-11-01 01:42,0.570,0.003,0,0,0,0,v\n2010-11-01 01:48,0.600,0.003,0,0,0,0,v\n2010-11-01 01:54,0.639,0.003,0,0,0,0,v\n2010-11-01 02:00,0.668,0.007,0,0,0,0,v\n2010-11-01 02:06,0.705,0.007,0,0,0,0,v\n2010-11-01 02:12,0.731,0.007,0,0,0,0,v\n2010

In [90]:
reader = csv.reader(response.text.splitlines())
columns = next(reader)

In [91]:
table_name = 'noaa_data'
print(columns)

column_defs = ', '.join([f'{col.strip().replace(" ","_").replace("(","").replace(")","")} TEXT' for col in columns]) 
print(column_defs)
sql = f'CREATE TABLE IF NOT EXISTS {table_name} ({column_defs})'
conn.execute(sql)
print("table created")
conn.commit()

['Date Time', ' Water Level', ' Sigma', ' O or I (for verified)', ' F', ' R', ' L', ' Quality ']
Date_Time TEXT, Water_Level TEXT, Sigma TEXT, O_or_I_for_verified TEXT, F TEXT, R TEXT, L TEXT, Quality TEXT
table created


In [66]:
for line in reader:
    print(line[0])
    placeholders = ", ".join([f"'{val}'" for val in line])
    print(placeholders)
    query = f'INSERT INTO {table_name} VALUES ({placeholders})'
    print(query)
    conn.execute(query)

2024-01-01 00:06
'2024-01-01 00:06', '2.043', '0.007', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-01 00:06', '2.043', '0.007', '0', '0', '0', '0', 'v')
2024-01-01 00:12
'2024-01-01 00:12', '2.043', '0.007', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-01 00:12', '2.043', '0.007', '0', '0', '0', '0', 'v')
2024-01-01 00:18
'2024-01-01 00:18', '2.046', '0.007', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-01 00:18', '2.046', '0.007', '0', '0', '0', '0', 'v')
2024-01-01 00:24
'2024-01-01 00:24', '2.017', '0.016', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-01 00:24', '2.017', '0.016', '0', '0', '0', '0', 'v')
2024-01-01 00:30
'2024-01-01 00:30', '2.010', '0.016', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-01 00:30', '2.010', '0.016', '0', '0', '0', '0', 'v')
2024-01-01 00:36
'2024-01-01 00:36', '2.010', '0.013', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-01 00:36', '2.010', '0

2024-01-01 15:06
'2024-01-01 15:06', '1.702', '0.036', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-01 15:06', '1.702', '0.036', '0', '0', '0', '0', 'v')
2024-01-01 15:12
'2024-01-01 15:12', '1.663', '0.039', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-01 15:12', '1.663', '0.039', '0', '0', '0', '0', 'v')
2024-01-01 15:18
'2024-01-01 15:18', '1.617', '0.030', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-01 15:18', '1.617', '0.030', '0', '0', '0', '0', 'v')
2024-01-01 15:24
'2024-01-01 15:24', '1.590', '0.026', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-01 15:24', '1.590', '0.026', '0', '0', '0', '0', 'v')
2024-01-01 15:30
'2024-01-01 15:30', '1.554', '0.036', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-01 15:30', '1.554', '0.036', '0', '0', '0', '0', 'v')
2024-01-01 15:36
'2024-01-01 15:36', '1.521', '0.033', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-01 15:36', '1.521', '0

2024-01-02 01:24
'2024-01-02 01:24', '2.135', '0.052', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-02 01:24', '2.135', '0.052', '0', '0', '0', '0', 'v')
2024-01-02 01:30
'2024-01-02 01:30', '2.109', '0.052', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-02 01:30', '2.109', '0.052', '0', '0', '0', '0', 'v')
2024-01-02 01:36
'2024-01-02 01:36', '2.073', '0.046', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-02 01:36', '2.073', '0.046', '0', '0', '0', '0', 'v')
2024-01-02 01:42
'2024-01-02 01:42', '2.046', '0.043', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-02 01:42', '2.046', '0.043', '0', '0', '0', '0', 'v')
2024-01-02 01:48
'2024-01-02 01:48', '2.027', '0.049', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-02 01:48', '2.027', '0.049', '0', '0', '0', '0', 'v')
2024-01-02 01:54
'2024-01-02 01:54', '2.004', '0.052', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-02 01:54', '2.004', '0

2024-01-02 15:00
'2024-01-02 15:00', '1.292', '0.046', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-02 15:00', '1.292', '0.046', '0', '0', '0', '0', 'v')
2024-01-02 15:06
'2024-01-02 15:06', '1.243', '0.049', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-02 15:06', '1.243', '0.049', '0', '0', '0', '0', 'v')
2024-01-02 15:12
'2024-01-02 15:12', '1.213', '0.043', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-02 15:12', '1.213', '0.043', '0', '0', '0', '0', 'v')
2024-01-02 15:18
'2024-01-02 15:18', '1.200', '0.043', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-02 15:18', '1.200', '0.043', '0', '0', '0', '0', 'v')
2024-01-02 15:24
'2024-01-02 15:24', '1.170', '0.046', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-02 15:24', '1.170', '0.046', '0', '0', '0', '0', 'v')
2024-01-02 15:30
'2024-01-02 15:30', '1.141', '0.049', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-02 15:30', '1.141', '0

2024-01-03 06:30
'2024-01-03 06:30', '1.377', '0.036', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-03 06:30', '1.377', '0.036', '0', '0', '0', '0', 'v')
2024-01-03 06:36
'2024-01-03 06:36', '1.354', '0.036', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-03 06:36', '1.354', '0.036', '0', '0', '0', '0', 'v')
2024-01-03 06:42
'2024-01-03 06:42', '1.344', '0.036', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-03 06:42', '1.344', '0.036', '0', '0', '0', '0', 'v')
2024-01-03 06:48
'2024-01-03 06:48', '1.318', '0.046', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-03 06:48', '1.318', '0.046', '0', '0', '0', '0', 'v')
2024-01-03 06:54
'2024-01-03 06:54', '1.305', '0.039', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-03 06:54', '1.305', '0.039', '0', '0', '0', '0', 'v')
2024-01-03 07:00
'2024-01-03 07:00', '1.292', '0.046', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-03 07:00', '1.292', '0

...

2024-01-29 11:48
'2024-01-29 11:48', '2.857', '0.049', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-29 11:48', '2.857', '0.049', '0', '0', '0', '0', 'v')
2024-01-29 11:54
'2024-01-29 11:54', '2.837', '0.052', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-29 11:54', '2.837', '0.052', '0', '0', '0', '0', 'v')
2024-01-29 12:00
'2024-01-29 12:00', '2.814', '0.049', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-29 12:00', '2.814', '0.049', '0', '0', '0', '0', 'v')
2024-01-29 12:06
'2024-01-29 12:06', '2.775', '0.043', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-29 12:06', '2.775', '0.043', '0', '0', '0', '0', 'v')
2024-01-29 12:12
'2024-01-29 12:12', '2.765', '0.043', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-29 12:12', '2.765', '0.043', '0', '0', '0', '0', 'v')
2024-01-29 12:18
'2024-01-29 12:18', '2.745', '0.049', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-29 12:18', '2.745', '0

2024-01-30 04:24
'2024-01-30 04:24', '1.745', '0.043', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-30 04:24', '1.745', '0.043', '0', '0', '0', '0', 'v')
2024-01-30 04:30
'2024-01-30 04:30', '1.718', '0.039', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-30 04:30', '1.718', '0.039', '0', '0', '0', '0', 'v')
2024-01-30 04:36
'2024-01-30 04:36', '1.708', '0.039', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-30 04:36', '1.708', '0.039', '0', '0', '0', '0', 'v')
2024-01-30 04:42
'2024-01-30 04:42', '1.676', '0.046', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-30 04:42', '1.676', '0.046', '0', '0', '0', '0', 'v')
2024-01-30 04:48
'2024-01-30 04:48', '1.669', '0.039', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-30 04:48', '1.669', '0.039', '0', '0', '0', '0', 'v')
2024-01-30 04:54
'2024-01-30 04:54', '1.646', '0.033', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-30 04:54', '1.646', '0

2024-01-30 16:48
'2024-01-30 16:48', '1.738', '0.013', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-30 16:48', '1.738', '0.013', '0', '0', '0', '0', 'v')
2024-01-30 16:54
'2024-01-30 16:54', '1.728', '0.013', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-30 16:54', '1.728', '0.013', '0', '0', '0', '0', 'v')
2024-01-30 17:00
'2024-01-30 17:00', '1.702', '0.016', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-30 17:00', '1.702', '0.016', '0', '0', '0', '0', 'v')
2024-01-30 17:06
'2024-01-30 17:06', '1.685', '0.013', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-30 17:06', '1.685', '0.013', '0', '0', '0', '0', 'v')
2024-01-30 17:12
'2024-01-30 17:12', '1.682', '0.013', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-30 17:12', '1.682', '0.013', '0', '0', '0', '0', 'v')
2024-01-30 17:18
'2024-01-30 17:18', '1.676', '0.013', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-30 17:18', '1.676', '0

INSERT INTO noaa_data VALUES ('2024-01-31 06:12', '1.889', '0.036', '0', '0', '0', '0', 'v')
2024-01-31 06:18
'2024-01-31 06:18', '1.892', '0.043', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-31 06:18', '1.892', '0.043', '0', '0', '0', '0', 'v')
2024-01-31 06:24
'2024-01-31 06:24', '1.895', '0.043', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-31 06:24', '1.895', '0.043', '0', '0', '0', '0', 'v')
2024-01-31 06:30
'2024-01-31 06:30', '1.902', '0.039', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-31 06:30', '1.902', '0.039', '0', '0', '0', '0', 'v')
2024-01-31 06:36
'2024-01-31 06:36', '1.909', '0.039', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-31 06:36', '1.909', '0.039', '0', '0', '0', '0', 'v')
2024-01-31 06:42
'2024-01-31 06:42', '1.918', '0.036', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-31 06:42', '1.918', '0.036', '0', '0', '0', '0', 'v')
2024-01-31 06:48
'2024-01-31 06:48', '1.932', '

2024-01-31 22:24
'2024-01-31 22:24', '2.942', '0.039', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-31 22:24', '2.942', '0.039', '0', '0', '0', '0', 'v')
2024-01-31 22:30
'2024-01-31 22:30', '2.962', '0.039', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-31 22:30', '2.962', '0.039', '0', '0', '0', '0', 'v')
2024-01-31 22:36
'2024-01-31 22:36', '2.991', '0.039', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-31 22:36', '2.991', '0.039', '0', '0', '0', '0', 'v')
2024-01-31 22:42
'2024-01-31 22:42', '3.017', '0.039', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-31 22:42', '3.017', '0.039', '0', '0', '0', '0', 'v')
2024-01-31 22:48
'2024-01-31 22:48', '3.034', '0.036', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-31 22:48', '3.034', '0.036', '0', '0', '0', '0', 'v')
2024-01-31 22:54
'2024-01-31 22:54', '3.063', '0.036', '0', '0', '0', '0', 'v'
INSERT INTO noaa_data VALUES ('2024-01-31 22:54', '3.063', '0

In [53]:
print(line)
print(line[0])
placeholders = ', '.join([val for val in line])
print(placeholders)                          

['2024-01-31 23:54', '3.201', '0.036', '0', '0', '0', '0', 'v']
2024-01-31 23:54
2024-01-31 23:54, 3.201, 0.036, 0, 0, 0, 0, v


In [57]:
data = reader
"""Inserts data into the SQLite table."""
placeholders = ', '.join([val for val in reader])
query = f'INSERT INTO {table_name} VALUES ({placeholders})'
print(query)
conn.executemany(query, data)

INSERT INTO noaa_data VALUES ()


OperationalError: near ")": syntax error

In [40]:

def create_table(conn, table_name, columns):
    """Creates a table in the SQLite database."""
    column_defs = ', '.join([f'{col.strip().replace(" ","_").replace("(","").replace(")","")} TEXT' for col in columns])
    print(column_defs)
    conn.execute(f'CREATE TABLE IF NOT EXISTS {table_name} ({column_defs})')

def insert_data(conn, table_name, data):
    """Inserts data into the SQLite table."""
    placeholders = ', '.join(['?' for _ in data[0]])
    query = f'INSERT INTO {table_name} VALUES ({placeholders})'
    conn.executemany(query, data)

def csv_string_to_table(conn, csv_string, table_name):
    """Converts a CSV string to a SQLite table."""
    reader = csv.reader(csv_string.splitlines())
    columns = next(reader) 
    create_table(conn, table_name, columns)
    insert_data(conn, table_name, reader)

# Example usage
csv_data = """
Name,Age,City
Alice,25,New York
Bob,30,London
"""
#csv_data = response.text

csv_string_to_table(conn, csv_data, 'sample_data')
conn.commit()


OperationalError: near ")": syntax error

In [70]:
# Convert the CSV data into a DataFrame
# df = pd.read_csv(response.text)
 # gives an error too long for windows
 

In [109]:
scur = conn.cursor()
scur.execute(f'''select date(Date_Time), * 
from {table_name}
limit 10''')

In [110]:
for row in scur.fetchall():
    print(row)

('2024-01-01', '2024-01-01 00:06', '2.043', '0.007', '0', '0', '0', '0', 'v')
('2024-01-01', '2024-01-01 00:12', '2.043', '0.007', '0', '0', '0', '0', 'v')
('2024-01-01', '2024-01-01 00:18', '2.046', '0.007', '0', '0', '0', '0', 'v')
('2024-01-01', '2024-01-01 00:24', '2.017', '0.016', '0', '0', '0', '0', 'v')
('2024-01-01', '2024-01-01 00:30', '2.010', '0.016', '0', '0', '0', '0', 'v')
('2024-01-01', '2024-01-01 00:36', '2.010', '0.013', '0', '0', '0', '0', 'v')
('2024-01-01', '2024-01-01 00:42', '2.014', '0.010', '0', '0', '0', '0', 'v')
('2024-01-01', '2024-01-01 00:48', '2.000', '0.013', '0', '0', '0', '0', 'v')
('2024-01-01', '2024-01-01 00:54', '1.991', '0.016', '0', '0', '0', '0', 'v')
('2024-01-01', '2024-01-01 01:00', '1.977', '0.016', '0', '0', '0', '0', 'v')


In [97]:
for col in scur.description:
    print(col[0])

Date_Time
Water_Level
Sigma
O_or_I_for_verified
F
R
L
Quality


In [122]:
for mth in range(1,13):
    print(mth)
for year in range(2021,2014,-1):
    print(year)

1
2
3
4
5
6
7
8
9
10
11
12
2021
2020
2019
2018
2017
2016
2015


In [151]:
### GET YEARS WORTH OF DATA
station = '8571892'
for year in range(2000,1996,-1):
    print(year)
    for month in range(12,0,-1):
        print(month,url(year,month,station))
        response = requests.get(url(year,month,station))

        reader = csv.reader(response.text.splitlines())
        columns = next(reader)

        # create table with columns, already done

        for line in reader:
            placeholders = ", ".join([f"'{val}'" for val in line])
            query = f'INSERT INTO {table_name} VALUES ({placeholders})'
            conn.execute(query)
    conn.commit()

2000
12 https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?product=water_level&application=NOS.COOPS.TAC.WL&begin_date=20001201&end_date=20001231&datum=MLLW&station=8571892&time_zone=GMT&units=english&format=csv
11 https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?product=water_level&application=NOS.COOPS.TAC.WL&begin_date=20001101&end_date=20001130&datum=MLLW&station=8571892&time_zone=GMT&units=english&format=csv
10 https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?product=water_level&application=NOS.COOPS.TAC.WL&begin_date=20001001&end_date=20001031&datum=MLLW&station=8571892&time_zone=GMT&units=english&format=csv
9 https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?product=water_level&application=NOS.COOPS.TAC.WL&begin_date=20000901&end_date=20000930&datum=MLLW&station=8571892&time_zone=GMT&units=english&format=csv
8 https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?product=water_level&application=NOS.COOPS.TAC.WL&begin_date=20000801&end_date=20

9 https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?product=water_level&application=NOS.COOPS.TAC.WL&begin_date=19970901&end_date=19970930&datum=MLLW&station=8571892&time_zone=GMT&units=english&format=csv
8 https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?product=water_level&application=NOS.COOPS.TAC.WL&begin_date=19970801&end_date=19970831&datum=MLLW&station=8571892&time_zone=GMT&units=english&format=csv
7 https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?product=water_level&application=NOS.COOPS.TAC.WL&begin_date=19970701&end_date=19970731&datum=MLLW&station=8571892&time_zone=GMT&units=english&format=csv
6 https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?product=water_level&application=NOS.COOPS.TAC.WL&begin_date=19970601&end_date=19970630&datum=MLLW&station=8571892&time_zone=GMT&units=english&format=csv
5 https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?product=water_level&application=NOS.COOPS.TAC.WL&begin_date=19970501&end_date=19970531&d

OperationalError: table noaa_data has 8 columns but 1 values were supplied

In [112]:
conn.commit()

In [137]:
scur = conn.cursor()
scur.execute(f'drop view if exists daily') 
scur.execute(f'''create view daily as 

select date(Date_Time) as report_date, 
max(water_level) as max_water_level,
min(water_level) as min_water_level,
count(*) as record_count
from {table_name}
group by 1
order by 1
''')


In [135]:
scur = conn.cursor()
scur.execute(f'''
select *
from daily
order by 1
limit 10
''')

for col in scur.description:
    print(col[0])

for row in scur.fetchall():
    print(row)

report_date
max_water_level
min_water_level
record_count
('2014-01-01', '2.056', '-0.024', 240)
('2014-01-02', '2.243', '-0.017', 240)
('2014-01-03', '2.286', '-0.001', 240)
('2014-01-04', '3.017', '-0.001', 240)
('2014-01-05', '2.985', '0.524', 240)
('2014-01-06', '2.663', '0.577', 240)
('2014-01-07', '2.365', '-0.014', 240)
('2014-01-08', '0.832', '-0.004', 240)
('2014-01-09', '0.819', '-0.004', 240)
('2014-01-10', '1.003', '-0.007', 240)


In [152]:
scur = conn.cursor()
scur.execute(f'''
select date(report_date,'start of year'),

avg(max_water_level), avg(min_water_level), max(max_water_level),
sum(record_count), count(*)
from daily
group by 1
order by 1
''')

for col in scur.description:
    print(col[0])

for row in scur.fetchall():
    print(row)

date(report_date,'start of year')
avg(max_water_level)
avg(min_water_level)
max(max_water_level)
sum(record_count)
count(*)
('1997-01-01', 2.204351020408163, 0.2956938775510201, '3.441', 58800, 245)
('1998-01-01', 2.344400000000001, 0.3492328767123284, '4.028', 87600, 365)
('1999-01-01', 2.1945342465753424, 0.2551041095890407, '3.811', 87600, 365)
('2000-01-01', 2.0724305555555564, 0.21978055555555526, '3.539', 86266, 360)
('2001-01-01', 2.042046575342468, 0.19994246575342445, '3.762', 87600, 365)
('2002-01-01', 2.0187890410958897, 0.17968493150684925, '3.559', 87600, 365)
('2003-01-01', 2.1190136986301384, 0.23942739726027368, '6.180', 87600, 365)
('2004-01-01', 2.1847205479452065, 0.2514328767123285, '3.976', 87600, 365)
('2005-01-01', 2.3458767123287654, 0.3540876712328764, '4.015', 87600, 365)
('2006-01-01', 2.2850438356164395, 0.30486301369863006, '3.903', 87600, 365)
('2007-01-01', 2.135452054794519, 0.25422465753424645, '3.706', 87600, 365)
('2008-01-01', 2.2406356164383547, 0.3

In [155]:
scur = conn.cursor()
scur.execute(f'''select 
 date(report_date,'start of year'),
 date(report_date,'start of month'),

avg(max_water_level), avg(min_water_level), max(max_water_level),
sum(record_count), count(*)
from daily
where date(report_date,'start of year') >= '2015-01-01'
group by 1,2
order by 1,2
''')

for col in scur.description:
    print(col[0])

for row in scur.fetchall():
    print(row)

date(report_date,'start of year')
date(report_date,'start of month')
avg(max_water_level)
avg(min_water_level)
max(max_water_level)
sum(record_count)
count(*)
('2015-01-01', '2015-01-01', 1.7364193548387097, 0.07106451612903227, '2.683', 7440, 31)
('2015-01-01', '2015-02-01', 1.8442857142857143, 0.14860714285714285, '3.496', 6720, 28)
('2015-01-01', '2015-03-01', 1.6970000000000005, 0.023, '2.653', 7440, 31)
('2015-01-01', '2015-04-01', 2.1169000000000002, 0.19136666666666666, '3.303', 7200, 30)
('2015-01-01', '2015-05-01', 2.147645161290322, 0.18054838709677415, '2.955', 7440, 31)
('2015-01-01', '2015-06-01', 2.5612, 0.37799999999999995, '3.605', 7200, 30)
('2015-01-01', '2015-07-01', 2.6617096774193545, 0.6055161290322579, '3.264', 7440, 31)
('2015-01-01', '2015-08-01', 2.5580645161290323, 0.5238064516129032, '3.293', 7440, 31)
('2015-01-01', '2015-09-01', 2.7270999999999996, 0.7227, '3.441', 7200, 30)
('2015-01-01', '2015-10-01', 2.7359354838709673, 0.6435483870967739, '3.834', 7440

In [148]:
scur = conn.cursor()
scur.execute(f'''
select date(Date_Time,'start of year'),

count(*)
from noaa_data
--where date(Date_Time,'start of year') = '2014-01-01'
group by 1
order by 1
''')

for col in scur.description:
    print(col[0])

for row in scur.fetchall():
    print(row)

date(Date_Time,'start of year')
count(*)
('2015-01-01', 87600)
('2016-01-01', 87600)
('2017-01-01', 87600)
('2018-01-01', 87600)
('2019-01-01', 87600)
('2020-01-01', 87600)
('2021-01-01', 87600)
('2022-01-01', 87600)
('2023-01-01', 87600)
('2024-01-01', 87599)
